## Covid-19 Number of Deaths prediction in the state of Kerala , India. 

### Importing and Initializaing Pyspark for jupyter NB

In [1]:
import findspark
findspark.init()



In [2]:
import os
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row

import collections
import datetime
import pyspark.sql.functions as F

In [3]:
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns
import squarify
import plotly.offline as py
import plotly_express as px

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA


from IPython.display import Image
warnings.filterwarnings('ignore')
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/Users/ashwinv/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### Builidng spark session

In [4]:
spark = SparkSession.builder.appName("CovidIndia").getOrCreate()

In [5]:
lines = spark.read.csv("/Users/ashwinv/Documents/SEM4/BigData/Project/Datasets/covidindia.csv", header=True, mode="DROPMALFORMED")


In [6]:
covidData = lines.select("State/UnionTerritory","Confirmed","Cured","Deaths")

In [8]:
pd1 = covidData.fillna("unknown")
pd1.show(50)

+--------------------+---------+-----+------+
|State/UnionTerritory|Confirmed|Cured|Deaths|
+--------------------+---------+-----+------+
|              Kerala|        1|    0|     0|
|              Kerala|        1|    0|     0|
|              Kerala|        2|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|    0|     0|
|              Kerala|        3|  

In [9]:
covidData.printSchema()

root
 |-- State/UnionTerritory: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Cured: string (nullable = true)
 |-- Deaths: string (nullable = true)



In [10]:
type(covidData)

pyspark.sql.dataframe.DataFrame

In [11]:
display()

In [12]:
covidData.describe().toPandas()

,summary,State/UnionTerritory,Confirmed,Cured,Deaths
0,count,9291,9291,9291,9291
1,mean,None,91839.78355397697,78632.66020880421,1487.620385319126
2,stddev,None,216601.36826584276,193110.2283333888,4713.81369001881
3,min,Andaman and Nicobar Islands,0,0,0
4,max,West Bengal,99930,99988,9984


In [13]:
ls

CovidindiaVisualization.ipynb


In [14]:
lines2 = spark.read.csv("/Users/ashwinv/Documents/SEM4/BigData/Project/Datasets/COVID-19CoronaVirusIndiaDataset/complete.csv", header=True, mode="DROPMALFORMED")

In [15]:
covidData2 = lines2.select("Date","Name of State / UT","Total Confirmed cases","Death","Cured/Discharged/Migrated")
cd2 = covidData2.fillna("unknown")

In [16]:
from datetime import datetime
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
cd2 = cd2.withColumn("Date",cd2['Date'].cast(DateType()))
cd2 = cd2.withColumn("Death",cd2['Death'].cast(IntegerType()))
cd2 = cd2.withColumn("Total Confirmed cases",cd2['Total Confirmed cases'].cast(IntegerType()))

In [17]:
cd2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Name of State / UT: string (nullable = false)
 |-- Total Confirmed cases: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Cured/Discharged/Migrated: string (nullable = false)



In [18]:
cd2.show()

+----------+------------------+---------------------+-----+-------------------------+
|      Date|Name of State / UT|Total Confirmed cases|Death|Cured/Discharged/Migrated|
+----------+------------------+---------------------+-----+-------------------------+
|2020-01-30|            Kerala|                    1|    0|                      0.0|
|2020-01-31|            Kerala|                    1|    0|                      0.0|
|2020-02-01|            Kerala|                    2|    0|                      0.0|
|2020-02-02|            Kerala|                    3|    0|                      0.0|
|2020-02-03|            Kerala|                    3|    0|                      0.0|
|2020-02-04|            Kerala|                    3|    0|                      0.0|
|2020-02-05|            Kerala|                    3|    0|                      0.0|
|2020-02-06|            Kerala|                    3|    0|                      0.0|
|2020-02-07|            Kerala|                    3| 

In [19]:
cd2.groupBy('Name of State / UT').max('Total Confirmed cases').show(50)

+--------------------+--------------------------+
|  Name of State / UT|max(Total Confirmed cases)|
+--------------------+--------------------------+
|            Nagaland|                      2498|
|           Karnataka|                    151449|
|              Odisha|                     39018|
|              Kerala|                     29151|
|              Ladakh|                      1592|
|Dadra and Nagar H...|                      1366|
|          Tamil Nadu|                    273460|
|           Telengana|                      4111|
|        Chhattisgarh|                     10407|
|      Andhra Pradesh|                    186461|
|      Madhya Pradesh|                     35734|
|              Punjab|                     19856|
|             Manipur|                      3093|
|                 Goa|                      7423|
|             Mizoram|                       537|
|    Himachal Pradesh|                      2916|
|          Puducherry|                      4433|


In [20]:
cd2gp = cd2.groupBy('Name of State / UT').max('Death')



In [21]:
cd2gp.show(40)


+--------------------+----------+
|  Name of State / UT|max(Death)|
+--------------------+----------+
|            Nagaland|         6|
|           Karnataka|      2804|
|              Odisha|       225|
|              Kerala|        94|
|              Ladakh|         7|
|Dadra and Nagar H...|         2|
|          Tamil Nadu|      4461|
|           Telengana|       156|
|        Chhattisgarh|        71|
|      Andhra Pradesh|      1681|
|      Madhya Pradesh|       929|
|              Punjab|       491|
|             Manipur|         7|
|                 Goa|        64|
|             Mizoram|         0|
|    Himachal Pradesh|        14|
|          Puducherry|        65|
|             Haryana|       455|
|   Jammu and Kashmir|       426|
|           Jharkhand|       136|
|        Telangana***|       455|
|   Arunachal Pradesh|         3|
|             Gujarat|      2556|
|              Sikkim|         1|
|               Delhi|      4044|
|          Chandigarh|        20|
|           Ra

In [22]:
from pyspark.sql.functions import col
cd2k = cd2.select('Name of State / UT','Total Confirmed cases','Date','Death').filter(col('Name of State / UT') == "Kerala")
cd2k.show(50)

+------------------+---------------------+----------+-----+
|Name of State / UT|Total Confirmed cases|      Date|Death|
+------------------+---------------------+----------+-----+
|            Kerala|                    1|2020-01-30|    0|
|            Kerala|                    1|2020-01-31|    0|
|            Kerala|                    2|2020-02-01|    0|
|            Kerala|                    3|2020-02-02|    0|
|            Kerala|                    3|2020-02-03|    0|
|            Kerala|                    3|2020-02-04|    0|
|            Kerala|                    3|2020-02-05|    0|
|            Kerala|                    3|2020-02-06|    0|
|            Kerala|                    3|2020-02-07|    0|
|            Kerala|                    3|2020-02-08|    0|
|            Kerala|                    3|2020-02-09|    0|
|            Kerala|                    3|2020-02-10|    0|
|            Kerala|                    3|2020-02-11|    0|
|            Kerala|                    

In [30]:
data= cd2k.select('Total Confirmed cases','Death')
data.show(3)

+---------------------+-----+
|Total Confirmed cases|Death|
+---------------------+-----+
|                    1|    0|
|                    1|    0|
|                    2|    0|
+---------------------+-----+
only showing top 3 rows



### Transforming Integer features to vectors.

In [31]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['Total Confirmed cases'], outputCol = 'features')
data = vectorAssembler.transform(data)
data = data.select(['features', 'Death'])

In [32]:
data

DataFrame[features: vector, Death: int]

### Splitting into test and train sets

In [36]:
splits = data.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

### Linear Regression Model

In [37]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Death', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0030966812496913123]
Intercept: 2.9961498569203764


In [38]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 3.668371
r2: 0.972918


In [39]:
train_df.describe().show()

+-------+------------------+
|summary|             Death|
+-------+------------------+
|  count|               128|
|   mean|         15.109375|
| stddev|22.378714005750666|
|    min|                 0|
|    max|                94|
+-------+------------------+



### Evaluating on test data

In [45]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","Death","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Death",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----+--------+
|        prediction|Death|features|
+------------------+-----+--------+
|2.9992465381700675|    0|   [1.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
+------------------+-----+--------+
only showing top 5 rows

R Squared (R2) on test data = 0.907918


In [46]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 3.87808


### Finding residual for each data point

In [50]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.tail(5)

numIterations: 2
objectiveHistory: [0.4999999999999999, 0.4040780248977257, 0.02529205901871722]


[Row(residuals=1.3857626632171645),
 Row(residuals=-1.23425771767198),
 Row(residuals=-2.213265079875015),
 Row(residuals=-2.566970873290714),
 Row(residuals=0.7324950333281777)]

In [52]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show(40)

numIterations: 2
objectiveHistory: [0.4999999999999999, 0.4040780248977257, 0.02529205901871722]
+-------------------+
|          residuals|
+-------------------+
|-2.9992465381700675|
| -3.002343219419759|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
|  -3.00543990066945|
| -3.020923306917907|
| -3.042600075665746|
|-3.0487934381651285|
|-3.0673735256632764|
| -3.082856931911733|
| -3.120017106908029|
|-3.1571772819043247|
| -3.290334575641051|
|-3.3615582443839513|
| -3.420395188128086|
|-3.5411657568660475|
|

In [54]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","Death","features").show()

+------------------+-----+--------+
|        prediction|Death|features|
+------------------+-----+--------+
|2.9992465381700675|    0|   [1.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
|  3.00543990066945|    0|   [3.0]|
| 3.024019988167598|    0|   [9.0]|
|3.0487934381651285|    0|  [17.0]|
|3.0487934381651285|    0|  [17.0]|
| 3.054986800664511|    0|  [19.0]|
|3.0673735256632764|    0|  [23.0]|
|3.0766635694123505|    0|  [26.0]|
|3.0797602506620416|    0|  [27.0]|
|3.0797602506620416|    0|  [27.0]|
| 3.203627500649694|    0|  [67.0]|
+------------------+-----+--------+
only showing top 20 rows



### Decision Tree Model

In [56]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'Death')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="Death", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 0.931161


In [57]:
dt_model.featureImportances

SparseVector(1, {0: 1.0})

### Gradient boosting Model

In [59]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'Death', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'Death', 'features').show(5)

+----------+-----+--------+
|prediction|Death|features|
+----------+-----+--------+
|       0.0|    0|   [1.0]|
|       0.0|    0|   [3.0]|
|       0.0|    0|   [3.0]|
|       0.0|    0|   [3.0]|
|       0.0|    0|   [3.0]|
+----------+-----+--------+
only showing top 5 rows



### Evaluaing the model

In [62]:
gbt_evaluator = RegressionEvaluator(
    labelCol="Death", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.914325


In [63]:
spark